<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/06_25_22_GosSbert_embeddings_extra_xgboost_politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U xgboost sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 65 kB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 1.8 MB 47.9 MB/s 
     |████████████████████████████████| 4.4 MB 50.0 MB/s 
     |████████████████████████████████| 1.2 MB 58.8 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.6 MB/s 
     |████████████████████████████████| 181 kB 65.8 MB/s 
     |████████████████████████████████| 145 kB 60.5 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.1-py3-none-any.whl size=125774 sha256=874c47f6ab19caa040c9cfbc1e6816d83066b36409460541f1725cffd1f3a847
  Stored in directory: /root/.cache/pip/wheels/58/27/2f/708b4f002c226e57b6243769da345c

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import xgboost as xgb

import wandb
from wandb.xgboost import WandbCallback

In [3]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return score_func.__name__.split("_")[0]


Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base_dir = Path("/content/drive/MyDrive/Thesis")
assert base_dir.exists()

In [7]:
df_fake = pd.read_csv(base_dir/'gossipcop_fake.csv')
df_real = pd.read_csv(base_dir/'gossipcop_real.csv')
df_fake['label']=0
df_real['label']=1
df_fake.count()

id           5323
news_url     5067
title        5323
tweet_ids    5135
label        5323
dtype: int64

In [8]:
df = df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [9]:
df.head()

,id,news_url,title,tweet_ids,label
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,0
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,0
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,0
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,0


Compute emebeddings

In [10]:
embedding_file = Path("/content/drive/MyDrive/sbert_embeddings.npy")

if embedding_file.exists():
    embeddings = np.load(embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)

    sentences = df.title.to_list()
    embeddings = model.encode(sentences, show_progress_bar=True)
    np.save(embedding_file, embeddings)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/692 [00:00<?, ?it/s]

Extra features: add more features here, if all looks good. 

Source

In [11]:
source = df.news_url.str.replace("^(https://)?web.archive.org/web/\d+/", "", regex=True)

In [12]:
source = source.str.replace("^(https?://)?(www.)?", "", regex=True)

In [13]:
source = source.str.replace("\.(com|info|org|gov|tv|us|news|me|co.uk|net|club|co|live|edu|xyz|site|life|ru|online|tk|website|pw|one|world|mil).*$", "", regex=True)

In [14]:
len(source), len(source.unique())


(22140, 2180)

Number of retweets

In [15]:
df["num_retweets"] = df.tweet_ids.fillna("").map(lambda x: len(str(x).split("\t")) if len(x) else 0)
df["log_num_retweets"] = np.log(df.num_retweets.to_numpy() + 1)

In [16]:
df.describe()

,label,num_retweets,log_num_retweets
count,22140.000000,22140.000000,22140.000000
mean,0.759575,66.843993,3.252656
std,0.427351,154.985377,1.472402
min,0.000000,0.000000,0.000000
25%,1.000000,11.000000,2.484907
50%,1.000000,38.000000,3.663562
75%,1.000000,64.000000,4.174387
max,1.000000,2568.000000,7.851272


More to come here..

Training

In [17]:
X = np.concatenate([embeddings, df.log_num_retweets.to_numpy()[..., None]], axis=1)
y = df.label.to_numpy()

X.shape, y.shape

((22140, 769), (22140,))

In [18]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [19]:
def train(train_idx, test_idx, params):

    # training
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain,'train'), (dtest,'eval')]
    clf = xgb.train(params, dtrain, num_boost_round=100, early_stopping_rounds=None, evals=watchlist, callbacks=[WandbCallback()])
    #evaluation
    probs = clf.predict(dtest)
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})

In [20]:
params = {
    "objective":'binary:logistic',
    "seed":124
}

In [21]:
for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    with wandb.init(entity="saloniteam", project="fnd", group="sbert-mpnet-v2+lnt-xgb", name=f"sbert-mpnet-v2+xgb-fold-{fold_id}") as run:
        train(train_idx, test_idx, params)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[0]	train-logloss:0.54266	eval-logloss:0.55334
[1]	train-logloss:0.45509	eval-logloss:0.47920
[2]	train-logloss:0.39666	eval-logloss:0.43092
[3]	train-logloss:0.35623	eval-logloss:0.40040
[4]	train-logloss:0.32454	eval-logloss:0.37695
[5]	train-logloss:0.29856	eval-logloss:0.36278
[6]	train-logloss:0.27811	eval-logloss:0.34943
[7]	train-logloss:0.26149	eval-logloss:0.34247
[8]	train-logloss:0.24679	eval-logloss:0.33421
[9]	train-logloss:0.23384	eval-logloss:0.32811
[10]	train-logloss:0.22202	eval-logloss:0.32551
[11]	train-logloss:0.21092	eval-logloss:0.32147
[12]	train-logloss:0.20267	eval-logloss:0.31862
[13]	train-logloss:0.19268	eval-logloss:0.31612
[14]	train-logloss:0.18606	eval-logloss:0.31357
[15]	train-logloss:0.17749	eval-logloss:0.31115
[16]	train-logloss:0.17115	eval-logloss:0.31050
[17]	train-logloss:0.16289	eval-logloss:0.30793
[18]	train-logloss:0.15735	eval-logloss:0.30587
[19]	train-logloss:0.15114	eval-logloss:0.30385
[20]	train-logloss:0.14688	eval-logloss:0.30256
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1,▁
precision,▁
recall,▁
train-logloss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.88392
epoch,99
f1,0.92559
precision,0.90183


wandb: Currently logged in as: saloni (saloniteam). Use `wandb login --relogin` to force relogin


...

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1,▁
precision,▁
recall,▁
train-logloss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.89589
epoch,99
f1,0.93343
precision,0.90736


[0]	train-logloss:0.54201	eval-logloss:0.55649
[1]	train-logloss:0.45472	eval-logloss:0.47993
[2]	train-logloss:0.39663	eval-logloss:0.43334
[3]	train-logloss:0.35440	eval-logloss:0.39989
[4]	train-logloss:0.32278	eval-logloss:0.37760
[5]	train-logloss:0.29825	eval-logloss:0.35960
[6]	train-logloss:0.27776	eval-logloss:0.34637
[7]	train-logloss:0.26082	eval-logloss:0.33608
[8]	train-logloss:0.24651	eval-logloss:0.32878
[9]	train-logloss:0.23309	eval-logloss:0.32296
[10]	train-logloss:0.22095	eval-logloss:0.31821
[11]	train-logloss:0.21114	eval-logloss:0.31447
[12]	train-logloss:0.20103	eval-logloss:0.31042
[13]	train-logloss:0.19158	eval-logloss:0.30714
[14]	train-logloss:0.18357	eval-logloss:0.30564
[15]	train-logloss:0.17623	eval-logloss:0.30310
[16]	train-logloss:0.16907	eval-logloss:0.30084
[17]	train-logloss:0.16320	eval-logloss:0.29844
[18]	train-logloss:0.15717	eval-logloss:0.29718
[19]	train-logloss:0.15117	eval-logloss:0.29497
[20]	train-logloss:0.14525	eval-logloss:0.29392
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1,▁
precision,▁
recall,▁
train-logloss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.89273
epoch,99
f1,0.93145
precision,0.90519


[0]	train-logloss:0.54326	eval-logloss:0.55538
[1]	train-logloss:0.45502	eval-logloss:0.48010
[2]	train-logloss:0.39636	eval-logloss:0.43012
[3]	train-logloss:0.35510	eval-logloss:0.39935
[4]	train-logloss:0.32276	eval-logloss:0.37654
[5]	train-logloss:0.29786	eval-logloss:0.36196
[6]	train-logloss:0.27885	eval-logloss:0.35022
[7]	train-logloss:0.26126	eval-logloss:0.34043
[8]	train-logloss:0.24678	eval-logloss:0.33238
[9]	train-logloss:0.23430	eval-logloss:0.32667
[10]	train-logloss:0.22302	eval-logloss:0.32097
[11]	train-logloss:0.21349	eval-logloss:0.31703
[12]	train-logloss:0.20327	eval-logloss:0.31416
[13]	train-logloss:0.19524	eval-logloss:0.31131
[14]	train-logloss:0.18850	eval-logloss:0.30961
[15]	train-logloss:0.17959	eval-logloss:0.30742
[16]	train-logloss:0.17270	eval-logloss:0.30552
[17]	train-logloss:0.16663	eval-logloss:0.30394
[18]	train-logloss:0.16074	eval-logloss:0.30283
[19]	train-logloss:0.15505	eval-logloss:0.30116
[20]	train-logloss:0.14959	eval-logloss:0.29961
[2

accuracy,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval-logloss,█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
f1,▁
precision,▁
recall,▁
train-logloss,█▆▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.89521
epoch,99
f1,0.93322
precision,0.90458
